# Import Needed Libraries

In [ ]:
from statistics import mean,stdev
import ndlib.models.ModelConfig as mc
from termcolor import colored
import ndlib.models.epidemics as ep
import collections
import heapq
import operator
import networkx as nx
import os
import os.path
import random
import numpy as np
from os import walk
from os import path
import sys
from utils import *
from variables import *

# Functions

In [ ]:
def get_top_k_best_nodes(dict_centrality, k):
    first_nodes = heapq.nlargest(k, dict_centrality, key=dict_centrality.get)
    return first_nodes

In [ ]:
def get_bottom_k_best_nodes(dict_centrality, k):
    bottom_nodes = heapq.nsmallest(k, dict_centrality, key=dict_centrality.get)
    return bottom_nodes

In [ ]:
def nth_moment(g,n):
    degree_np = np.array(list(dict(g.degree).values()))
    return (sum(degree_np**n)/len(g))

# SIR Simulations for all Networks

In [ ]:
networks = next(walk(RealNetworks), (None, None, []))[2]

for network in networks:
    counter = 0
    g = nx.read_gml(f'{RealNetworks}/{network}',destringizer=int)
    n = network.split(".")[0]

    print(colored(f'Simulation currently on network {network} : {nx.info(g)}...', 'red',attrs=['bold']))
    #Remove Self Loops
    g.remove_edges_from(nx.selfloop_edges(g))

    #Take Largest Component
    nb_components = nx.number_connected_components(g)
    if(nb_components>1):
        subgraphs = list(nx.connected_components(g))
        max = subgraphs[0]
        for subgraph in subgraphs:
            if len(subgraph)>len(max):
                max = subgraph
        g = g.subgraphs(max)
    #print(network)

    #Initialize a dictionary for each centrality
    list_dicts = []

    for centrality in colors.keys():
        globals()[f'dict_{centrality}'] = dict()
        list_dicts.append(globals()[f'dict_{centrality}'])

    #get centrality results
    for dictionary,centrality in zip(list_dicts,colors.keys()):
        p = f'/RealNetworks/centrality_results/{n}/dict_{centrality}.txt'
        with open(p) as raw_data:
            for item in raw_data:
                if ':' in item:
                    key,value = item.split(':', 1)
                    value = value.replace('\n', '')
                    dictionary[int(key)]=float(value)
                else:
                    pass # deal with bad lines of text here

    #Initialize Average and Standard Deviation for Each Centrality
    for centrality in colors.keys():
        globals()[f'{centrality}_average_each_simulation'] = dict()
        globals()[f'{centrality}_standard_each_simulation'] = dict()
        for fraction in fractions:
            globals()[f'{centrality}_average_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standard_each_simulation'][fraction] = 0

    p = f'./Pickle Variables/{n}/'
    if(path.exists(p)==False):
        os.mkdir(p)
    network_size = len(g.nodes())
    save_obj(network_size,p,'networkSize')

    #Epidemic threshold
    firstMoment = nth_moment(g,1)
    secondMoment = nth_moment(g,2)
    epidemicThreshold = round(firstMoment/(secondMoment-firstMoment),3)
    save_obj(epidemicThreshold,p,'epidemicThreshold')

    gamma = 0.1

    for centrality in colors.keys():
        print(colored(f'Centrality: {centrality}', 'green',attrs=['bold']))
        for fraction in fractions:
            globals()[f'l_{centrality}'] = list()
            fraction_infected = round(fraction*network_size)
            if(centrality=="modv_neg"):
                infected_nodes = get_bottom_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
            else:
                infected_nodes = get_top_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)

            #print(centrality,":",fraction)

            for i in range(repeatedSimulations):

                model = ep.SIRModel(g)
                config = mc.Configuration()
                config.add_model_parameter('beta', epidemicThreshold)
                config.add_model_parameter('gamma', gamma)
                config.add_model_initial_configuration("Infected", infected_nodes)
                model.set_initial_status(config)

                x= 0
                iterations = model.iteration_bunch(1)

                while((iterations[x]['node_count'][1])!=0):
                    x = x + 1
                    iterations = iterations + model.iteration_bunch(1)

                finalRecoveredInOneIteration = iterations[x]['node_count'][2]
                globals()[f'l_{centrality}'].append(finalRecoveredInOneIteration)

            globals()[f'{centrality}_average_each_simulation'][fraction] = mean(globals()[f'l_{centrality}'])
            globals()[f'{centrality}_standard_each_simulation'][fraction] = stdev(globals()[f'l_{centrality}'])
            print(fraction,":",mean(globals()[f'l_{centrality}']))

        p = f"./Pickle Variables/{n}/beta-{epidemicThreshold}_gamma-{gamma}/"
        if(path.exists(p)==False):
            os.mkdir(p)
        save_obj(globals()[f'{centrality}_average_each_simulation'], p,f'{centrality}_average_each_simulation')
        save_obj(globals()[f'{centrality}_standard_each_simulation'], p, f'{centrality}_standard_each_simulation')

Simulation currently on network networkovermu035.gml : Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 10417
Average degree:   8.3336...
Centrality: modv_pos
0.01 : 2468.96
0.02 : 2469.94
0.03 : 2471.63
0.04 : 2471.54
0.05 : 2469.79
0.06 : 2472.65
0.07 : 2473.27
0.08 : 2474.64
0.09 : 2474.75
0.1 : 2475.42
0.11 : 2477.13
0.12 : 2477
0.13 : 2477.63
0.14 : 2478.64
0.15 : 2479.36
0.16 : 2479.2
0.17 : 2478.27
0.18 : 2479.38
0.19 : 2480.64
0.2 : 2480.23
0.21 : 2480.71
0.22 : 2479.17
0.23 : 2480.22
0.24 : 2480.04
0.25 : 2480.97
0.26 : 2480.92
0.27 : 2481.7
0.28 : 2480.19
0.29 : 2480.37
0.3 : 2480.59
0.31 : 2480.67
0.32 : 2481.29
0.33 : 2481.81
0.34 : 2481.97
0.35 : 2482.94
0.36 : 2482.22
0.37 : 2483.07
0.38 : 2482.92
0.39 : 2483.12
0.4 : 2484.38
0.41 : 2483.71
0.42 : 2484.73
0.43 : 2484.6
0.44 : 2484.92
0.45 : 2485.18
0.46 : 2485.04
0.47 : 2484.9
0.48 : 2484.88
0.49 : 2486.03
0.5 : 2484.48
Centrality: com
0.01 : 2470.75
0.02 : 2471.01
0.03 : 2470.98
0.04 : 2470.89
0.05 : 2469.5
0.06

0.22 : 2469.17
0.23 : 2469.33
0.24 : 2468.68
0.25 : 2468.9
0.26 : 2469.61
0.27 : 2469.2
0.28 : 2468.96
0.29 : 2469.29
0.3 : 2469.65
0.31 : 2469.71
0.32 : 2470.31
0.33 : 2469.67
0.34 : 2469.18
0.35 : 2470.9
0.36 : 2469.91
0.37 : 2470.73
0.38 : 2470.9
0.39 : 2469.67
0.4 : 2471.45
0.41 : 2470.33
0.42 : 2470.23
0.43 : 2470.36
0.44 : 2470.99
0.45 : 2469.83
0.46 : 2472.1
0.47 : 2471.75
0.48 : 2471.01
0.49 : 2471.86
0.5 : 2472.42
Simulation currently on network networkovermu030.gml : Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 10283
Average degree:   8.2264...
Centrality: modv_pos
0.01 : 2473.4
0.02 : 2473.43
0.03 : 2474.06
0.04 : 2473.85
0.05 : 2474.81
0.06 : 2474.25
0.07 : 2474.63
0.08 : 2475.63
0.09 : 2475.12
0.1 : 2476.48
0.11 : 2477.7
0.12 : 2478.01
0.13 : 2479.72
0.14 : 2479.43
0.15 : 2479.94
0.16 : 2479.93
0.17 : 2480.53
0.18 : 2481.21
0.19 : 2481.46
0.2 : 2482.19
0.21 : 2483.53
0.22 : 2483.11
0.23 : 2483.2
0.24 : 2483.27
0.25 : 2484.25
0.26 : 2483.66
0.27 : 2483.84
0.28 

0.44 : 2473.89
0.45 : 2474.92
0.46 : 2474.33
0.47 : 2474.95
0.48 : 2474.03
0.49 : 2475.9
0.5 : 2475.17
Centrality: map_eq
0.01 : 2471.41
0.02 : 2472.16
0.03 : 2472.61
0.04 : 2472.04
0.05 : 2472.16
0.06 : 2472.11
0.07 : 2471.45
0.08 : 2472.95
0.09 : 2471.61
0.1 : 2472.41
0.11 : 2471.78
0.12 : 2472.73
0.13 : 2471.11
0.14 : 2472.94
0.15 : 2472.86
0.16 : 2472.55
0.17 : 2471.93
0.18 : 2471.92
0.19 : 2471.45
0.2 : 2472.62
0.21 : 2472.8
0.22 : 2471.29
0.23 : 2472.25
0.24 : 2472.09
0.25 : 2471.47
0.26 : 2472.34
0.27 : 2471.16
0.28 : 2472.1
0.29 : 2471.78
0.3 : 2471.25
0.31 : 2471.98
0.32 : 2471.98
0.33 : 2472.65
0.34 : 2473.5
0.35 : 2471.91
0.36 : 2471.25
0.37 : 2471.83
0.38 : 2473.64
0.39 : 2472.33
0.4 : 2472.62
0.41 : 2472.66
0.42 : 2473.79
0.43 : 2474.16
0.44 : 2474.1
0.45 : 2472.98
0.46 : 2474.93
0.47 : 2474.31
0.48 : 2476.05
0.49 : 2474.25
0.5 : 2474.25
Simulation currently on network networkovermu015.gml : Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 10090
Average degree:   

0.15 : 2454.07
0.16 : 2453.66
0.17 : 2452.67
0.18 : 2452.53
0.19 : 2456.27
0.2 : 2452.69
0.21 : 2453.51
0.22 : 2452.68
0.23 : 2454.55
0.24 : 2454.34
0.25 : 2455.64
0.26 : 2456.66
0.27 : 2455.88
0.28 : 2456.85
0.29 : 2456.25
0.3 : 2455.85
0.31 : 2458.1
0.32 : 2459.75
0.33 : 2458.61
0.34 : 2458.91
0.35 : 2460.11
0.36 : 2459.4
0.37 : 2461.17
0.38 : 2462.37
0.39 : 2461.46
0.4 : 2465.25
0.41 : 2463.18
0.42 : 2466.3
0.43 : 2466.87
0.44 : 2466.3
0.45 : 2467.38
0.46 : 2467.65
0.47 : 2466.94
0.48 : 2466.99
0.49 : 2466.63
0.5 : 2467.97
Centrality: map_eq
0.01 : 2453.54
0.02 : 2454.24
0.03 : 2455.67
0.04 : 2453.35
0.05 : 2454.75
0.06 : 2451.97
0.07 : 2454.1
0.08 : 2452.65
0.09 : 2455.76
0.1 : 2454.23
0.11 : 2454.14
0.12 : 2454.03
0.13 : 2454.61
0.14 : 2453.19
0.15 : 2455.27
0.16 : 2454.02
0.17 : 2453.55
0.18 : 2455.98
0.19 : 2452.89
0.2 : 2456.02
0.21 : 2455.48
0.22 : 2454.34
0.23 : 2452.42
0.24 : 2455.05
0.25 : 2454.92
0.26 : 2454.97
0.27 : 2454.44
0.28 : 2460.19
0.29 : 2457.86
0.3 : 2455.91
0.3

0.38 : 2482.91
0.39 : 2485.03
0.4 : 2485.93
0.41 : 2486.48
0.42 : 2486.31
0.43 : 2486.88
0.44 : 2487.73
0.45 : 2486.65
0.46 : 2487.79
0.47 : 2488.11
0.48 : 2488.64
0.49 : 2489.65
0.5 : 2490.66
Centrality: degree
0.01 : 2343.2
0.02 : 2342.73
0.03 : 2345.84
0.04 : 2351.21
0.05 : 2343.97
0.06 : 2344.39
0.07 : 2345.52
0.08 : 2346.09
0.09 : 2342.53
0.1 : 2344.47
0.11 : 2348.28
0.12 : 2344.48
0.13 : 2345
0.14 : 2345.02
0.15 : 2343.44
0.16 : 2350.83
0.17 : 2350.87
0.18 : 2351.78
0.19 : 2350.37
0.2 : 2347.35
0.21 : 2347.24
0.22 : 2344.82
0.23 : 2348.43
0.24 : 2350.13
0.25 : 2348.35
0.26 : 2351.48
0.27 : 2352.66
0.28 : 2354.74
0.29 : 2367.86
0.3 : 2370.9
0.31 : 2371.68
0.32 : 2369.4
0.33 : 2377.54
0.34 : 2380.92
0.35 : 2383.8
0.36 : 2390.79
0.37 : 2397.54
0.38 : 2402.93
0.39 : 2395.87
0.4 : 2397.27
0.41 : 2401.11
0.42 : 2408.99
0.43 : 2410.85
0.44 : 2417.27
0.45 : 2418.01
0.46 : 2421.87
0.47 : 2422.44
0.48 : 2426.39
0.49 : 2426.31
0.5 : 2426.54
Centrality: map_eq
0.01 : 2343.68
0.02 : 2345.69
0

0.1 : 2476.83
0.11 : 2476.21
0.12 : 2478.5
0.13 : 2478
0.14 : 2477.27
0.15 : 2477.97
0.16 : 2477.95
0.17 : 2477.79
0.18 : 2478.18
0.19 : 2479.39
0.2 : 2478.55
0.21 : 2477.91
0.22 : 2478.62
0.23 : 2480.35
0.24 : 2479.9
0.25 : 2478.81
0.26 : 2480.42
0.27 : 2478.62
0.28 : 2480.26
0.29 : 2479.91
0.3 : 2481.66
0.31 : 2480.69
0.32 : 2480.58
0.33 : 2481.09
0.34 : 2480.81
0.35 : 2481.42
0.36 : 2482.4
0.37 : 2481.75
0.38 : 2483.62
0.39 : 2482.19
0.4 : 2482.61
0.41 : 2482.29
0.42 : 2483.54
0.43 : 2483.53
0.44 : 2482.68
0.45 : 2483.52
0.46 : 2483.32
0.47 : 2483.39
0.48 : 2484.21
0.49 : 2484.45
0.5 : 2483.9
Centrality: degree
0.01 : 2472.8
0.02 : 2472.44
0.03 : 2472.44
0.04 : 2471.36
0.05 : 2473.75
0.06 : 2471.84
0.07 : 2471.58
0.08 : 2471.65
0.09 : 2471.96
0.1 : 2472.08
0.11 : 2472.2
0.12 : 2472.81
0.13 : 2471.16
0.14 : 2471.85
0.15 : 2473.71
0.16 : 2472.57
0.17 : 2472.84
0.18 : 2471.43
0.19 : 2471.2
0.2 : 2473.5
0.21 : 2471.76
0.22 : 2471.86
0.23 : 2472.46
0.24 : 2471.85
0.25 : 2472.06
0.26 : 24

0.34 : 2429.1
0.35 : 2429.77
0.36 : 2426.61
0.37 : 2430.85
0.38 : 2425.62
0.39 : 2429.13
0.4 : 2430.04
0.41 : 2429.91
0.42 : 2429.73
0.43 : 2430.83
0.44 : 2432.43
0.45 : 2431.84
0.46 : 2433.85
0.47 : 2441.54
0.48 : 2440.59
0.49 : 2441.78
0.5 : 2441.39
Centrality: modv_abs
0.01 : 1947.56
0.02 : 1981.03
0.03 : 1979.91
0.04 : 1978.36
0.05 : 1983.41
0.06 : 1980.39
0.07 : 1981.03
0.08 : 2011.82
0.09 : 2026.58
0.1 : 2049.77
0.11 : 2093.07
0.12 : 2130.84
0.13 : 2151.97
0.14 : 2174.4
0.15 : 2203.04
0.16 : 2207.54
0.17 : 2228.56
0.18 : 2239.83
0.19 : 2257.92
0.2 : 2278.47
0.21 : 2306.86
0.22 : 2324.97
0.23 : 2326.21
0.24 : 2328.94
0.25 : 2328.13
0.26 : 2328.42
0.27 : 2344.88
0.28 : 2353.35
0.29 : 2376.68
0.3 : 2386.81
0.31 : 2396.56
0.32 : 2412.73
0.33 : 2413.07
0.34 : 2413.67
0.35 : 2414.93
0.36 : 2416.28
0.37 : 2420.43
0.38 : 2428.72
0.39 : 2432.05
0.4 : 2438.09
0.41 : 2446.98
0.42 : 2448.1
0.43 : 2452.04
0.44 : 2455.33
0.45 : 2454.82
0.46 : 2455.84
0.47 : 2455
0.48 : 2455.12
0.49 : 2456.28
0

0.04 : 2475.37
0.05 : 2476.33
0.06 : 2476.39
0.07 : 2476.18
0.08 : 2476.11
0.09 : 2476.89
0.1 : 2476.07
0.11 : 2476.75
0.12 : 2476.89
0.13 : 2476.27
0.14 : 2475.41
0.15 : 2476.47
0.16 : 2476.92
0.17 : 2477.1
0.18 : 2476.85
0.19 : 2478.3
0.2 : 2477.75
0.21 : 2478.36
0.22 : 2477.72
0.23 : 2479.4
0.24 : 2479.36
0.25 : 2480.13
0.26 : 2480.26
0.27 : 2480.19
0.28 : 2482.14
0.29 : 2480.76
0.3 : 2482.09
0.31 : 2483.28
0.32 : 2483.14
0.33 : 2483.75
0.34 : 2482.9
0.35 : 2483.24
0.36 : 2483.59
0.37 : 2483.13
0.38 : 2483.82
0.39 : 2484.07
0.4 : 2483.8
0.41 : 2483.88
0.42 : 2484.52
0.43 : 2484.4
0.44 : 2484.89
0.45 : 2484.24
0.46 : 2484.21
0.47 : 2484.67
0.48 : 2485.56
0.49 : 2484.86
0.5 : 2484.39
Centrality: modv_abs
0.01 : 2476.26
0.02 : 2475.49
0.03 : 2475.33
0.04 : 2476.3
0.05 : 2476.66
0.06 : 2477.17
0.07 : 2475.72
0.08 : 2476.61
0.09 : 2475.8
0.1 : 2476.31
0.11 : 2477.1
0.12 : 2477.2
0.13 : 2478.07
0.14 : 2478.45
0.15 : 2478.96
0.16 : 2478.99
0.17 : 2479.26
0.18 : 2479.38
0.19 : 2479.61
0.2 :

0.27 : 2468.31
0.28 : 2469.36
0.29 : 2470.45
0.3 : 2470.07
0.31 : 2470.98
0.32 : 2470.7
0.33 : 2471.28
0.34 : 2471.57
0.35 : 2471.75
0.36 : 2472.22
0.37 : 2470.18
0.38 : 2471.61
0.39 : 2472.15
0.4 : 2472.96
0.41 : 2473.46
0.42 : 2472.98
0.43 : 2474.46
0.44 : 2473.69
0.45 : 2474.74
0.46 : 2474.2
0.47 : 2475.99
0.48 : 2477.06
0.49 : 2477.02
0.5 : 2477.71
Centrality: modv_neg
0.01 : 2467.82
0.02 : 2467.81
0.03 : 2466.64
0.04 : 2468.27
0.05 : 2466.76
0.06 : 2466.07
0.07 : 2467.78
0.08 : 2465.77
0.09 : 2467.59
0.1 : 2467.58
0.11 : 2468.95
0.12 : 2469.05
0.13 : 2466.74
0.14 : 2467.29
0.15 : 2469.65
0.16 : 2469.34
0.17 : 2470.56
0.18 : 2471.76
0.19 : 2472.52
0.2 : 2471.77
0.21 : 2473.28
0.22 : 2474.01
0.23 : 2474.95
0.24 : 2475.82
0.25 : 2475.81
0.26 : 2474.54
0.27 : 2475.54
0.28 : 2476.92
0.29 : 2476.95
0.3 : 2477.27
0.31 : 2477.32
0.32 : 2477.11
0.33 : 2478.02
0.34 : 2478.68
0.35 : 2479.19
0.36 : 2478.77
0.37 : 2479.48
0.38 : 2479.32
0.39 : 2479.76
0.4 : 2480.43
0.41 : 2479.77
0.42 : 2480.1

0.49 : 2477.23
0.5 : 2478.29
Centrality: cbc
0.01 : 2466.51
0.02 : 2465.02
0.03 : 2465.39
0.04 : 2464.86
0.05 : 2466.56
0.06 : 2466.07
0.07 : 2464.77
0.08 : 2467.58
0.09 : 2465.68
0.1 : 2465.9
0.11 : 2465.95
0.12 : 2465.88
0.13 : 2465.15
0.14 : 2467.89
0.15 : 2466.34
0.16 : 2467.74
0.17 : 2464.78
0.18 : 2468.16
0.19 : 2466.43
0.2 : 2466.86
0.21 : 2467.88
0.22 : 2466.67
0.23 : 2468.1
0.24 : 2468.21
0.25 : 2468.28
0.26 : 2467.18
0.27 : 2470.1
0.28 : 2469.02
0.29 : 2468.54
0.3 : 2470.56
0.31 : 2468.68
0.32 : 2469.77
0.33 : 2469.52
0.34 : 2470.8
0.35 : 2469.73
0.36 : 2470.45
0.37 : 2472.03
0.38 : 2470.79
0.39 : 2473.08
0.4 : 2473.48
0.41 : 2474.44
0.42 : 2474.21
0.43 : 2474.36
0.44 : 2474.28
0.45 : 2476.02
0.46 : 2476.97
0.47 : 2475.09
0.48 : 2477.18
0.49 : 2478.19
0.5 : 2476.76
Centrality: modv_neg
0.01 : 2466.1
0.02 : 2464.68
0.03 : 2465.29
0.04 : 2465.15
0.05 : 2465.3
0.06 : 2466.14
0.07 : 2465.17
0.08 : 2465.22
0.09 : 2466.08
0.1 : 2465.28
0.11 : 2467.89
0.12 : 2467.43
0.13 : 2467.59
0